# Working with fraud datasets for EDA

In [ ]:
import glob
import numpy as np
import pandas as pd
import pyspark
import urllib
import pylab as pl
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.feature import *

In [ ]:
spark = SparkSession.builder.appName('eda').getOrCreate()

In [ ]:
# Downloaded a bunch of datasets from kaggle / similar
glob.glob("dataset/*")

In [ ]:
# inferSchema reads the file twice, but detects numerical columns
data = spark.read.csv('dataset/creditcard_2023.csv', header=True, inferSchema=True)
data.printSchema()

## Clustering

In [ ]:
data

In [ ]:
# create feature column

feature_cols = data.columns[1:-2]
assembler = pyspark.ml.feature.VectorAssembler(inputCols=feature_cols, outputCol='features')
data = assembler.transform(data)
data.show(10)

In [ ]:
data = data.select(['features', 'Class'])
data.show(10)

In [ ]:
km = pyspark.ml.clustering.KMeans(k=3)
xs = data.select(['features'])
clustering = km.fit(xs)
labels = clustering.transform(xs)

In [ ]:
labels.groupBy('prediction').count().show()

In [ ]:
labels.select(['prediction']).summary().show(10)

In [ ]:
train, test = data.randomSplit([0.8, 0.2])

In [ ]:
train.summary().show()

In [ ]:
test.summary().show()

## PCA

In [ ]:
xs = data.select(['features'])

scaler = StandardScaler(
    inputCol = 'features', 
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
).fit(xs)

xs_scaled = scaler.transform(xs)
xs_scaled.show(6)

In [ ]:
n_components = 2
pca = PCA(
    k = n_components, 
    inputCol = 'scaledFeatures', 
    outputCol = 'pcaFeatures'
).fit(xs_scaled)

xs_pca = pca.transform(xs_scaled)
print('Explained Variance Ratio', pca.explainedVariance.toArray())
xs_pca.show(6)

In [ ]:
# pull back to "local machine"
local_xs = xs_pca.rdd.map(lambda row: row.pcaFeatures).collect()
local_xs = np.array(local_xs)

In [ ]:
local_y = np.array(data.select('Class').collect()).ravel()

In [ ]:
pl.scatter(local_xs[:, 0], local_xs[:, 1], s=8, alpha=0.3, c=local_y)
pl.tight_layout()

## Second dataset

In [ ]:
data = spark.read.csv('dataset/detect/creditcard_train.csv', header=True, inferSchema=True)
data.printSchema()

In [ ]:
feature_cols = data.columns[1:-1]
assembler = pyspark.ml.feature.VectorAssembler(inputCols=feature_cols, outputCol='features')
data = assembler.transform(data)
data.show(10)

In [ ]:
xs = data.select(['features'])

scaler = StandardScaler(
    inputCol = 'features', 
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
).fit(xs)

xs_scaled = scaler.transform(xs)
xs_scaled.show(6)

n_components = 2
pca = PCA(
    k = n_components, 
    inputCol = 'scaledFeatures', 
    outputCol = 'pcaFeatures'
).fit(xs_scaled)

xs_pca = pca.transform(xs_scaled)
print('Explained Variance Ratio', pca.explainedVariance.toArray())
xs_pca.show(6)

# pull back to "local machine"
local_xs = xs_pca.rdd.map(lambda row: row.pcaFeatures).collect()
local_xs = np.array(local_xs)

local_y = np.array(data.select('Class').collect()).ravel()

In [ ]:
pl.scatter(local_xs[:, 0], local_xs[:, 1], s=8, alpha=0.3, c=local_y)
pl.tight_layout()

## Third dataset
More complex, text features

In [ ]:
data = spark.read.csv('dataset/insurance/train.csv', header=True, inferSchema=True)
data.printSchema()